In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

In [2]:
# matching format from https://gist.github.com/DGalt/6b419549086d3aadff8b7cc072fc60b3
complaint_df = pd.read_csv('data/complaints-cpd-2016-oct.csv')
print(complaint_df.dtypes)
complaint_df.head()

COMPLAINT_NUMBER                    object
INCIDENT_DATE                       object
COMPLAINT_DATE                      object
CLOSED_DATE                         object
ACCUSEDOFFICER_FNAME                object
ACCUSEDOFFICER_LNAME                object
ACCUSEDOFFICER_FULL_NAME            object
ACCUSEDOFFICER_UNIT                 object
ACCUSEDOFFICER_APPOINTMENT_DATE     object
INITIAL_CATEGORY_CODE               object
INITIAL_CATEGORY                    object
CURRENT_CATEGORY_CODE               object
CURRENT_CATEGORY                    object
ACTION_CODE                        float64
ACTION_DESCRIPTION                  object
FINDING_CODE                        object
dtype: object


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,COMPLAINT_NUMBER,INCIDENT_DATE,COMPLAINT_DATE,CLOSED_DATE,ACCUSEDOFFICER_FNAME,ACCUSEDOFFICER_LNAME,ACCUSEDOFFICER_FULL_NAME,ACCUSEDOFFICER_UNIT,ACCUSEDOFFICER_APPOINTMENT_DATE,INITIAL_CATEGORY_CODE,INITIAL_CATEGORY,CURRENT_CATEGORY_CODE,CURRENT_CATEGORY,ACTION_CODE,ACTION_DESCRIPTION,FINDING_CODE
0,237004,21-MAY-1997,21-MAY-1997,25-MAY-1999,ANTHONY,ABBATE,"ABBATE, ANTHONY G",11,05-DEC-1994,05A,ARRESTEE - DURING ARREST,05B,ARRESTEE - DURING ARREST,600.0,NO ACTION TAKEN,UN
1,258124,19-NOV-1999,20-NOV-1999,07-DEC-2000,ANTHONY,ABBATE,"ABBATE, ANTHONY G",11,05-DEC-1994,04E,PRISONERS PROPERTY - INVENTORY/RECEIPT,04E,PRISONERS PROPERTY - INVENTORY/RECEIPT,600.0,NO ACTION TAKEN,NS
2,190369,11-FEB-1992,11-FEB-1992,09-APR-1992,CARMEL,ABBATE,"ABBATE, CARMEL G",641,06-JAN-1969,03G,MISCELLANEOUS,03G,MISCELLANEOUS,600.0,NO ACTION TAKEN,UN
3,210426,30-JUL-1994,14-JUL-1994,01-DEC-1994,CARMEL,ABBATE,"ABBATE, CARMEL G",640,06-JAN-1969,10V,INVENTORY PROCEDURES,04E,INVENTORY PROCEDURES,600.0,NO ACTION TAKEN,NS
4,219424,12-JUL-1995,19-JUL-1995,23-JAN-1996,CARMEL,ABBATE,"ABBATE, CARMEL G",640,06-JAN-1969,09J,MISCELLANEOUS,05A,MISCELLANEOUS,600.0,NO ACTION TAKEN,NS


In [15]:
complaint_df['INCIDENT_DATE'] = pd.to_datetime(complaint_df['INCIDENT_DATE'])
complaint_df['COMPLAINT_DATE'] = pd.to_datetime(complaint_df['COMPLAINT_DATE'])
complaint_df['CLOSED_DATE'] = pd.to_datetime(complaint_df['CLOSED_DATE'])
complaint_df['ACCUSEDOFFICER_APPOINTMENT_DATE'] = pd.to_datetime(complaint_df['ACCUSEDOFFICER_APPOINTMENT_DATE'])
complaint_df.head()

,COMPLAINT_NUMBER,INCIDENT_DATE,COMPLAINT_DATE,CLOSED_DATE,ACCUSEDOFFICER_FNAME,ACCUSEDOFFICER_LNAME,ACCUSEDOFFICER_FULL_NAME,ACCUSEDOFFICER_UNIT,ACCUSEDOFFICER_APPOINTMENT_DATE,INITIAL_CATEGORY_CODE,INITIAL_CATEGORY,CURRENT_CATEGORY_CODE,CURRENT_CATEGORY,ACTION_CODE,ACTION_DESCRIPTION,FINDING_CODE
0,237004,1997-05-21,1997-05-21,1999-05-25,ANTHONY,ABBATE,"ABBATE, ANTHONY G",11,1994-12-05,05A,ARRESTEE - DURING ARREST,05B,ARRESTEE - DURING ARREST,600.0,NO ACTION TAKEN,UN
1,258124,1999-11-19,1999-11-20,2000-12-07,ANTHONY,ABBATE,"ABBATE, ANTHONY G",11,1994-12-05,04E,PRISONERS PROPERTY - INVENTORY/RECEIPT,04E,PRISONERS PROPERTY - INVENTORY/RECEIPT,600.0,NO ACTION TAKEN,NS
2,190369,1992-02-11,1992-02-11,1992-04-09,CARMEL,ABBATE,"ABBATE, CARMEL G",641,1969-01-06,03G,MISCELLANEOUS,03G,MISCELLANEOUS,600.0,NO ACTION TAKEN,UN
3,210426,1994-07-30,1994-07-14,1994-12-01,CARMEL,ABBATE,"ABBATE, CARMEL G",640,1969-01-06,10V,INVENTORY PROCEDURES,04E,INVENTORY PROCEDURES,600.0,NO ACTION TAKEN,NS
4,219424,1995-07-12,1995-07-19,1996-01-23,CARMEL,ABBATE,"ABBATE, CARMEL G",640,1969-01-06,09J,MISCELLANEOUS,05A,MISCELLANEOUS,600.0,NO ACTION TAKEN,NS


In [7]:
context_df = pd.read_csv('data/complaints-cpd-2016-oct-context-data.csv')
print(context_df.dtypes)
context_df.head()

ACCUSEDOFFICER_FNAME                object
ACCUSEDOFFICER_LNAME                object
ACCUSEDOFFICER_FULL_NAME            object
ACCUSEDOFFICER_UNIT                 object
ACCUSEDOFFICER_APPOINTMENT_DATE     object
ORIGIN_APPOINTMENT_DATE             object
FIRST_NME                           object
LAST_NME                            object
SEX_CODE_CD                         object
RACE                                object
DOBYEAR                            float64
STATUS_I                            object
APPOINTED_DATE                      object
EMPLOYEE_POSITION_CD                object
DESCR                               object
CPD_UNIT_ASSIGNED_NO               float64
UNITDESCR                           object
RESIGNATION_DATE                    object
STAR1                              float64
STAR2                              float64
STAR3                              float64
STAR4                              float64
STAR5                              float64
STAR6      

,ACCUSEDOFFICER_FNAME,ACCUSEDOFFICER_LNAME,ACCUSEDOFFICER_FULL_NAME,ACCUSEDOFFICER_UNIT,ACCUSEDOFFICER_APPOINTMENT_DATE,ORIGIN_APPOINTMENT_DATE,FIRST_NME,LAST_NME,SEX_CODE_CD,RACE,...,STAR1,STAR2,STAR3,STAR4,STAR5,STAR6,STAR7,STAR8,STAR9,STAR10
0,ANTHONY,ABBATE,"ABBATE, ANTHONY G",11.0,05-DEC-94,05-DEC-1994,ANTHONY,ABBATE,M,WHITE,...,18601.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CARMEL,ABBATE,"ABBATE, CARMEL G",641.0,06-JAN-69,06-JAN-1969,CARMEL,ABBATE,M,WHITE,...,20484.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CARMEN,ABBATE,"ABBATE, CARMEN S",25.0,16-OCT-54,16-OCT-1954,CARMEN,ABBATE,M,WHITE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TERRY,ABBATE,"ABBATE, TERRY M",12.0,04-DEC-95,04-DEC-1995,TERRY,ABBATE,M,WHITE,...,19596.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MICHAEL,ABBEY,"ABBEY, MICHAEL J",151.0,26-APR-76,26-APR-1976,MICHAEL,ABBEY,M,BLACK,...,16511.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
context_df_merge = context_df[['ACCUSEDOFFICER_FULL_NAME', 'ACCUSEDOFFICER_APPOINTMENT_DATE', 
                               'SEX_CODE_CD', 'RACE', 'DOBYEAR']]
context_df_merge['ACCUSEDOFFICER_APPOINTMENT_DATE'] = pd.to_datetime(context_df_merge['ACCUSEDOFFICER_APPOINTMENT_DATE'])
context_df_merge['DOBYEAR'] = context_df_merge['DOBYEAR'].apply(lambda x: str(int(x)) if not pd.isnull(x) else None)
context_df_merge.head()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,ACCUSEDOFFICER_FULL_NAME,ACCUSEDOFFICER_APPOINTMENT_DATE,SEX_CODE_CD,RACE,DOBYEAR
0,"ABBATE, ANTHONY G",1994-12-05,M,WHITE,1968
1,"ABBATE, CARMEL G",1969-01-06,M,WHITE,1942
2,"ABBATE, CARMEN S",2054-10-16,M,WHITE,1930
3,"ABBATE, TERRY M",1995-12-04,M,WHITE,1972
4,"ABBEY, MICHAEL J",1976-04-26,M,BLACK,1952


In [16]:
merge_format_df = complaint_df.merge(context_df_merge, 
                                     on=['ACCUSEDOFFICER_FULL_NAME', 'ACCUSEDOFFICER_APPOINTMENT_DATE'],
                                     how='left')
merge_format_df.head()

,COMPLAINT_NUMBER,INCIDENT_DATE,COMPLAINT_DATE,CLOSED_DATE,ACCUSEDOFFICER_FNAME,ACCUSEDOFFICER_LNAME,ACCUSEDOFFICER_FULL_NAME,ACCUSEDOFFICER_UNIT,ACCUSEDOFFICER_APPOINTMENT_DATE,INITIAL_CATEGORY_CODE,INITIAL_CATEGORY,CURRENT_CATEGORY_CODE,CURRENT_CATEGORY,ACTION_CODE,ACTION_DESCRIPTION,FINDING_CODE,SEX_CODE_CD,RACE,DOBYEAR
0,237004,1997-05-21,1997-05-21,1999-05-25,ANTHONY,ABBATE,"ABBATE, ANTHONY G",11,1994-12-05,05A,ARRESTEE - DURING ARREST,05B,ARRESTEE - DURING ARREST,600.0,NO ACTION TAKEN,UN,M,WHITE,1968
1,258124,1999-11-19,1999-11-20,2000-12-07,ANTHONY,ABBATE,"ABBATE, ANTHONY G",11,1994-12-05,04E,PRISONERS PROPERTY - INVENTORY/RECEIPT,04E,PRISONERS PROPERTY - INVENTORY/RECEIPT,600.0,NO ACTION TAKEN,NS,M,WHITE,1968
2,190369,1992-02-11,1992-02-11,1992-04-09,CARMEL,ABBATE,"ABBATE, CARMEL G",641,1969-01-06,03G,MISCELLANEOUS,03G,MISCELLANEOUS,600.0,NO ACTION TAKEN,UN,M,WHITE,1942
3,210426,1994-07-30,1994-07-14,1994-12-01,CARMEL,ABBATE,"ABBATE, CARMEL G",640,1969-01-06,10V,INVENTORY PROCEDURES,04E,INVENTORY PROCEDURES,600.0,NO ACTION TAKEN,NS,M,WHITE,1942
4,219424,1995-07-12,1995-07-19,1996-01-23,CARMEL,ABBATE,"ABBATE, CARMEL G",640,1969-01-06,09J,MISCELLANEOUS,05A,MISCELLANEOUS,600.0,NO ACTION TAKEN,NS,M,WHITE,1942


In [17]:
merge_format_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 232270 entries, 0 to 232269
Data columns (total 19 columns):
COMPLAINT_NUMBER                   232270 non-null object
INCIDENT_DATE                      232208 non-null datetime64[ns]
COMPLAINT_DATE                     232270 non-null datetime64[ns]
CLOSED_DATE                        231554 non-null datetime64[ns]
ACCUSEDOFFICER_FNAME               232270 non-null object
ACCUSEDOFFICER_LNAME               232270 non-null object
ACCUSEDOFFICER_FULL_NAME           232270 non-null object
ACCUSEDOFFICER_UNIT                124909 non-null object
ACCUSEDOFFICER_APPOINTMENT_DATE    232085 non-null datetime64[ns]
INITIAL_CATEGORY_CODE              232270 non-null object
INITIAL_CATEGORY                   232270 non-null object
CURRENT_CATEGORY_CODE              232251 non-null object
CURRENT_CATEGORY                   232270 non-null object
ACTION_CODE                        228627 non-null float64
ACTION_DESCRIPTION                 228627 no

In [18]:
merge_df = merge_format_df[['COMPLAINT_NUMBER', 'INCIDENT_DATE',
                           'COMPLAINT_DATE', 'CLOSED_DATE', 
                           'ACCUSEDOFFICER_FULL_NAME', 'DOBYEAR',
                           'SEX_CODE_CD', 'RACE', 'ACCUSEDOFFICER_APPOINTMENT_DATE',
                           'ACCUSEDOFFICER_UNIT', 'CURRENT_CATEGORY', 'FINDING_CODE',
                           'ACTION_DESCRIPTION']].copy()
merge_df.head()

,COMPLAINT_NUMBER,INCIDENT_DATE,COMPLAINT_DATE,CLOSED_DATE,ACCUSEDOFFICER_FULL_NAME,DOBYEAR,SEX_CODE_CD,RACE,ACCUSEDOFFICER_APPOINTMENT_DATE,ACCUSEDOFFICER_UNIT,CURRENT_CATEGORY,FINDING_CODE,ACTION_DESCRIPTION
0,237004,1997-05-21,1997-05-21,1999-05-25,"ABBATE, ANTHONY G",1968,M,WHITE,1994-12-05,11,ARRESTEE - DURING ARREST,UN,NO ACTION TAKEN
1,258124,1999-11-19,1999-11-20,2000-12-07,"ABBATE, ANTHONY G",1968,M,WHITE,1994-12-05,11,PRISONERS PROPERTY - INVENTORY/RECEIPT,NS,NO ACTION TAKEN
2,190369,1992-02-11,1992-02-11,1992-04-09,"ABBATE, CARMEL G",1942,M,WHITE,1969-01-06,641,MISCELLANEOUS,UN,NO ACTION TAKEN
3,210426,1994-07-30,1994-07-14,1994-12-01,"ABBATE, CARMEL G",1942,M,WHITE,1969-01-06,640,INVENTORY PROCEDURES,NS,NO ACTION TAKEN
4,219424,1995-07-12,1995-07-19,1996-01-23,"ABBATE, CARMEL G",1942,M,WHITE,1969-01-06,640,MISCELLANEOUS,NS,NO ACTION TAKEN


In [19]:
merge_df = merge_df.rename(columns={'COMPLAINT_NUMBER': 'Complaint_Number',
                                    'INCIDENT_DATE': 'Incident_Datetime',
                                    'COMPLAINT_DATE': 'Complaint_Date',
                                    'CLOSED_DATE': 'Closed_Date',
                                    'ACCUSEDOFFICER_FULL_NAME': 'Accused_Name',
                                    'DOBYEAR': 'Accused_Birth_Yr',
                                    'SEX_CODE_CD': 'Accused_Gender',
                                    'RACE': 'Accused_Race_Code',
                                    'ACCUSEDOFFICER_APPOINTMENT_DATE': 'Accused_Date_Appointed',
                                    'ACCUSEDOFFICER_UNIT': 'Accused_Current_Unit',
                                    'CURRENT_CATEGORY': 'Accused_Complaint_Category',
                                    'FINDING_CODE': 'Accused_Final_Finding',
                                    'ACTION_DESCRIPTION': 'Accused_Discipline'})
merge_df.head()

,Complaint_Number,Incident_Datetime,Complaint_Date,Closed_Date,Accused_Name,Accused_Birth_Yr,Accused_Gender,Accused_Race_Code,Accused_Date_Appointed,Accused_Current_Unit,Accused_Complaint_Category,Accused_Final_Finding,Accused_Discipline
0,237004,1997-05-21,1997-05-21,1999-05-25,"ABBATE, ANTHONY G",1968,M,WHITE,1994-12-05,11,ARRESTEE - DURING ARREST,UN,NO ACTION TAKEN
1,258124,1999-11-19,1999-11-20,2000-12-07,"ABBATE, ANTHONY G",1968,M,WHITE,1994-12-05,11,PRISONERS PROPERTY - INVENTORY/RECEIPT,NS,NO ACTION TAKEN
2,190369,1992-02-11,1992-02-11,1992-04-09,"ABBATE, CARMEL G",1942,M,WHITE,1969-01-06,641,MISCELLANEOUS,UN,NO ACTION TAKEN
3,210426,1994-07-30,1994-07-14,1994-12-01,"ABBATE, CARMEL G",1942,M,WHITE,1969-01-06,640,INVENTORY PROCEDURES,NS,NO ACTION TAKEN
4,219424,1995-07-12,1995-07-19,1996-01-23,"ABBATE, CARMEL G",1942,M,WHITE,1969-01-06,640,MISCELLANEOUS,NS,NO ACTION TAKEN


In [22]:
merge_df = merge_df.sort_values(by='Complaint_Date')
merge_df.head()

,Complaint_Number,Incident_Datetime,Complaint_Date,Closed_Date,Accused_Name,Accused_Birth_Yr,Accused_Gender,Accused_Race_Code,Accused_Date_Appointed,Accused_Current_Unit,Accused_Complaint_Category,Accused_Final_Finding,Accused_Discipline
79610,35735,1967-10-21,1967-10-21,1980-08-01,"MURRAY, HUGH",None,NaN,NaN,NaT,NaN,THEFT,SU,RESIGNED -NOT SERVED
66817,35735,1967-10-21,1967-10-21,1980-08-01,"LYNCH, FRANK P",NaN,NaN,NaN,1953-09-21,171,THEFT,SU,30 DAY SUSPENSION
117247,35735,1967-10-21,1967-10-21,1980-08-01,"WALSH, THOMAS F",None,NaN,NaN,NaT,NaN,THEFT,SU,RESIGNED -NOT SERVED
81764,39347,1968-06-04,1968-06-05,1973-04-15,"NUCCIO, RICHARD",None,NaN,NaN,NaT,NaN,"""""U"""" NUMBER CONVERTED TO C.R. (INFORMATION ONLY)",SU,SEPARATION
90413,41480,1968-10-15,1968-10-15,1968-10-31,"POTE, LOUIS C",None,NaN,NaN,NaT,NaN,ABUSE OF AUTHORITY,SU,SEPARATION


In [23]:
merge_df.to_csv('data/matching-format-cpd-oct-complaints.csv',index=False)